In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_path = "drive/MyDrive/232E/"

In [ ]:
!pip install igraph

!pip install cairocffi

In [ ]:
from igraph import *
import json
import random
import numpy as np
from numpy import linalg 
import itertools
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay  # needed for triangulation
import os
import pandas as pd
from copy import deepcopy
from tqdm import tqdm
import networkx as nx
import pandas as pd
import igraph

In [ ]:
random.seed(2022)
np.random.seed(2022)

In [ ]:
with open(file_path+'los_angeles_censustracts.json') as f:
    census_tracts = json.loads(f.readline())

In [ ]:
display_names = dict()
coordinates = dict()

for area in census_tracts['features']:
    id = int(area['properties']['MOVEMENT_ID'])
    display_name = area['properties']['DISPLAY_NAME']
    display_names[id] = display_name
    a = area['geometry']['coordinates'][0]
    coordinates[id] = np.array(a if type(a[0][0]) == float else a[0]).mean(axis=0)

In [ ]:
g = Graph(directed=False)
g.add_vertices(len(display_names))
g.vs['display_name'] = list(display_names.values())  # index = id - 1
g.vs['coordinates'] = list(coordinates.values())

In [ ]:
month_filter = {12}  # for monthly aggregate data of 4th quarter, we can filter data based off of only December

edges = []
weights = []

with open(file_path+'los_angeles-censustracts-2019-4-All-MonthlyAggregate.csv') as f:
    f.readline()  # skip the first line
    
    while True:
        line = f.readline()
        if line == '':
            break  # end of file
        
        vals = line.strip().split(',')
        
        # read edge info
        src, dest, month, dist = int(vals[0]), int(vals[1]), int(vals[2]), float(vals[3])

        # if data is not relevant, skip it
        if month not in month_filter:
            continue
            
        edges.append((src - 1, dest - 1))
        weights.append(dist)  

In [ ]:
g.add_edges(edges)
g.es['weight'] = weights
del edges, weights

In [ ]:
print(len(g.vs), len(g.es))

In [ ]:
# keep only the giant connected component
components = g.components()
gcc = max(components, key=len)
vs_to_delete = [i for i in range(len(g.vs)) if i not in gcc]
g.delete_vertices(vs_to_delete)

# remove duplicate edges
g = g.simplify(combine_edges=dict(weight='mean'))  # combine duplicate edges

In [ ]:
coordinates = np.array(g.vs['coordinates'])
tri = Delaunay(g.vs['coordinates'])

In [ ]:
plt.figure(figsize=(15,15))
plt.triplot(coordinates[:, 0], coordinates[:, 1], tri.simplices.copy())
Bbox = ((-118.75,-117.75,33.7,34.4))
la_image = plt.imread(file_path+"Los_Angeles_Open_Streetmap.png")
plt.title("Road mesh of LA using Delaunay Triangualation algorithm")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.xlim(Bbox[0],Bbox[1])
plt.ylim(Bbox[2],Bbox[3])
plt.imshow(la_image,zorder=0,extent=Bbox,aspect='equal')
plt.show()


In [ ]:
edges_to_induce = []
for i in tqdm(range(tri.simplices.shape[0])):
    for col1,col2 in ((0,1),(1,2),(0,2)):
        v1,v2 = tri.simplices[i,col1],tri.simplices[i,col2] 
        bool_value = g.are_connected(v1,v2)
        if(bool_value):
            eid = g.get_eid(v1,v2)
            edges_to_induce.append(eid)

In [ ]:
tri_g = g.subgraph_edges(edges_to_induce)

In [ ]:
plt.figure(figsize=(15,15))
for e in tqdm(tri_g.es):
    start = e.source
    end = e.target
    v1 = np.array(tri_g.vs(start)['coordinates'])
    v2 = np.array(tri_g.vs(end)['coordinates'])
    data = np.vstack([v1,v2])
    x = data[:, 0]
    y = data[:, 1]
    plt.plot(x, y, 'blue')
plt.axis('equal')
Bbox = ((-118.75,-117.75,33.7,34.4))
la_image = plt.imread(file_path+"Los_Angeles_Open_Streetmap.png")
plt.title("Road mesh after removing fake edges")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.xlim(Bbox[0],Bbox[1])
plt.ylim(Bbox[2],Bbox[3])
plt.imshow(la_image,zorder=0,extent= Bbox,aspect='equal')
plt.show()

In [ ]:
def create_graph_with_euclid_distance(graph):
    edge_list = graph.es()
    node_list = graph.vs()
    for i in edge_list:
        node_0 = node_list(i.source)
        node_1 = node_list(i.target)
        coord0 = np.array(node_0['coordinates']).squeeze()
        coord1 = np.array(node_1['coordinates']).squeeze()
        distance = np.linalg.norm(coord0-coord1,2)
        i['euclidean_cost']=distance
create_graph_with_euclid_distance(tri_g)

In [ ]:
tri_g.summary()

In [ ]:
import matplotlib.pyplot as plt
weights = tri_g.es['weight']
binwidth = 20
bins = np.arange(min(weights), max(weights) + binwidth, binwidth)
plt.hist(weights, bins=bins)
plt.xlabel("Travelling Time")
plt.ylabel("Frequency")
plt.title("Frequency vs Travelling Time")
plt.show()


In [ ]:
visual_style = {}
visual_style["vertex_size"]=3
plot(tri_g,**visual_style)

### Estimating Traffic Flows Q15
### Max Flow Analysis

In [ ]:
edge_ends = np.array(tri_g.es['weight'])/3600
distances= 69*np.array(tri_g.es['euclidean_cost'])
speeds =  distances/edge_ends
car_length = 0.003  # (miles)
safety_distances = car_length+speeds * (2/3600)  # in miles, derived from 2 sec distance
cars_per_mile =  distances/(safety_distances)
n_lanes = 2
cars_per_hour = n_lanes*cars_per_mile*speeds

In [ ]:
tri_g.es['capacity'] = cars_per_hour

In [ ]:
source_coordinates= [-118.78,34.026]
dest_coordinates =  [-118.18,33.77]

malibu = np.asarray(source_coordinates)
long_beach = np.asarray(dest_coordinates) 


In [ ]:
#Now that we have the capacities of each road sorted out

closest_to_malibu = np.inf 
closest_to_long_beach = np.inf
malibu_vertex = 0
long_beach_vertex = 0
for i in tqdm(range(0,len(coordinates))):
    coord = coordinates[i]
    distance_2_malibu = np.linalg.norm(coord-malibu,2)
    distance_2_longbeach = np.linalg.norm(coord-long_beach,2)
    if(distance_2_malibu<closest_to_malibu):
        closest_to_malibu = distance_2_malibu
        malibu_vertex = i
    if(distance_2_longbeach<closest_to_long_beach):
        closest_to_long_beach = distance_2_longbeach
        long_beach_vertex = i


In [ ]:
print(malibu_vertex)

In [ ]:
print(long_beach_vertex)

In [ ]:
print('Number of edge-disjoint paths: ',tri_g.adhesion(long_beach_vertex,malibu_vertex))

In [ ]:
print(tri_g.maxflow(malibu_vertex,long_beach_vertex,capacity=tri_g.es["capacity"]))

In [ ]:
plt.figure(figsize=(10,10))
plt.triplot(coordinates[:, 0], coordinates[:, 1], tri.simplices.copy())
Bbox = ((-118.75,-117.75,33.7,34.4))
la_image = plt.imread(file_path+"Los_Angeles_Open_Streetmap.png")

plt.ylim(coordinates[malibu_vertex][1]-0.03,coordinates[malibu_vertex][1]+0.03)
plt.xlim(coordinates[malibu_vertex][0]-0.03,coordinates[malibu_vertex][0]+0.03)
plt.plot(coordinates[malibu_vertex][0],coordinates[malibu_vertex][1],marker="*",color='red')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Road map near Malibu')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.triplot(coordinates[:, 0], coordinates[:, 1], tri.simplices.copy())
Bbox = ((-118.75,-117.75,33.7,34.4))
plt.ylim(coordinates[long_beach_vertex][1]-0.03,coordinates[long_beach_vertex][1]+0.03)
plt.xlim(coordinates[long_beach_vertex][0]-0.03,coordinates[long_beach_vertex][0]+0.03)
plt.plot(coordinates[long_beach_vertex][0],coordinates[long_beach_vertex][1],marker="*",color='red')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Road map near Long Beach')
plt.show()

In [ ]:
trim_thresh = 800

In [ ]:
edges_to_induce = tri_g.es.select(weight_le=trim_thresh)
tri_g_trimmed = tri_g.subgraph_edges(edges_to_induce)

In [ ]:
plt.figure(figsize=(15,15), dpi=200)
for e in tri_g.es:
    v1 = tri_g.vs[e.source]['coordinates']
    v2 = tri_g.vs[e.target]['coordinates']
    data = np.vstack([v1, v2])
    x = data[:, 0]
    y = data[:, 1]
    plt.plot(x, y, 'blue' if e['weight'] < trim_thresh else 'red')
Bbox = ((-118.75,-117.75,33.7,34.4))
la_image = plt.imread(file_path+"Los_Angeles_Open_Streetmap.png")
plt.title("Road mesh of LA after Trimming")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.xlim(Bbox[0],Bbox[1])
plt.ylim(Bbox[2],Bbox[3])
plt.imshow(la_image,zorder=0,extent=Bbox,aspect='equal')
plt.show()


In [ ]:
visual_style = {}
visual_style["vertex_size"] = 3
plot(tri_g_trimmed,"Trimmed_G.png",**visual_style)

### Trimmed graph 

In [ ]:
print('Number of edge-disjoint paths: ',tri_g_trimmed.adhesion(long_beach_vertex,malibu_vertex))
print('Degree Distribution of nodes (Malibu, Long Beach): ', tri_g_trimmed.degree(malibu_vertex,mode='out')-1,tri_g_trimmed.degree(long_beach_vertex,mode='in')-1)

In [ ]:
print(tri_g_trimmed.maxflow(malibu_vertex,long_beach_vertex,capacity=tri_g_trimmed.es["capacity"]))

In [ ]:
coordinates = np.array(tri_g_trimmed.vs['coordinates'])

In [ ]:
plt.figure(figsize=(15,15), dpi=200)
for e in tri_g_trimmed.es:
    v1 = tri_g_trimmed.vs[e.source]['coordinates']
    v2 = tri_g_trimmed.vs[e.target]['coordinates']
    data = np.vstack([v1, v2])
    x = data[:, 0]
    y = data[:, 1]
    plt.plot(x, y,"blue")

plt.title("Road Map near Malibu")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.ylim(coordinates[malibu_vertex][1]-0.03,coordinates[malibu_vertex][1]+0.03)
plt.xlim(coordinates[malibu_vertex][0]-0.03,coordinates[malibu_vertex][0]+0.03)
plt.plot(coordinates[malibu_vertex][0],coordinates[malibu_vertex][1],marker="*",color='red')
plt.show()


In [ ]:
plt.figure(figsize=(15,15), dpi=200)
for e in tri_g_trimmed.es:
    v1 = tri_g_trimmed.vs[e.source]['coordinates']
    v2 = tri_g_trimmed.vs[e.target]['coordinates']
    data = np.vstack([v1, v2])
    x = data[:, 0]
    y = data[:, 1]
    plt.plot(x, y,"blue")

plt.title("Road Map near Malibu")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.ylim(coordinates[long_beach_vertex][1]-0.03,coordinates[long_beach_vertex][1]+0.03)
plt.xlim(coordinates[long_beach_vertex][0]-0.03,coordinates[long_beach_vertex][0]+0.03)
plt.plot(coordinates[long_beach_vertex][0],coordinates[long_beach_vertex][1],marker="*",color='red')
plt.show()


In [ ]:
def get_euclidean_distance(node_list):
    dist = np.zeros((len(node_list),len(node_list)))
    i= 0
    for node1 in (node_list):
        j=0
        for node2 in node_list:
            coord = np.array(node1['coordinates'])
            coord1 = np.array(node2['coordinates'])
            dist[i,j] = np.linalg.norm(coord1-coord,2)
            j+=1
        i+=1
    return dist

In [ ]:
euclid_distance = get_euclidean_distance(list(tri_g_trimmed.vs()))

In [ ]:
def get_distance(graph):
    vertex_seq = list(graph.vs())
    distance = np.zeros((len(vertex_seq),len(vertex_seq)))
    all_distances = np.array([i['euclidean_cost'] for i in graph.es()])
    counter_1 = 0
    counter_2  = 0
    for node1 in (vertex_seq): 
        all_paths=  graph.get_shortest_paths(node1,vertex_seq,weights=graph.es['euclidean_cost'],output='epath')
        counter_2 = 0
        for route in all_paths:
            local_distance=np.sum(all_distances[route])
            distance[counter_1,counter_2]=local_distance
            counter_2+=1
        counter_1+=1
    return distance

In [ ]:
def k_largest_index_argsort(a, k):
    idx = np.argsort(a.ravel())[:-k-1:-1]
    return np.column_stack(np.unravel_index(idx, a.shape))

### Strategy 1

In [ ]:
tri_g_trimmed.summary()

In [ ]:
normal_distance = get_distance(tri_g_trimmed)

In [ ]:
extra_distance = normal_distance - euclid_distance

top_20_indices = k_largest_index_argsort(extra_distance,50)

In [ ]:
index_list = set()
count = 0
while len(index_list)<20:
    i = top_20_indices[count]
    x = max(i[0],i[1])
    y = min(i[0],i[1])
    index_list.add((x,y))
    count+=1

index_list = list(index_list)
for i in index_list:
  print(i)

In [ ]:
tri_g_trimmed.is_connected()

In [ ]:
tri_with_new_edges_v1 = deepcopy(tri_g_trimmed)
for i in index_list:
    tri_with_new_edges_v1.add_edges([(i[0],i[1])])

In [ ]:
visual_style = {}
visual_style["vertex_size"] = 3
plot(tri_with_new_edges_v1,**visual_style)

In [ ]:
# plot the roads and highlight our new edges. 
plt.figure(figsize=(15,15), dpi=200)
for e in tqdm(tri_with_new_edges_v1.es):
    start = e.source
    end = e.target
    cur_edge = (max(start,end),min(start,end))

    v1 = np.array(tri_with_new_edges_v1.vs(start)['coordinates'])
    v2 = np.array(tri_with_new_edges_v1.vs(end)['coordinates'])
    data = np.vstack([v1,v2])
    x = data[:, 0]
    y = data[:, 1]
    if cur_edge in index_list:
        plt.plot(x,y,"red") 
    else:
        plt.plot(x, y,'blue')
# mark the source and destination
# source = tri_g.vs.select(display_name=source_address)[0].index
# target = tri_g.vs.select(display_name=dest_address)[0].index
# v1 = road_map_static.vs[source_idx]['coordinates']
# v2 = road_map_static.vs[dest_idx]['coordinates']
# data = np.vstack([v1, v2])
# x = data[:, 0]
# y = data[:, 1]
# plt.plot(x, y, 'bo')
plt.axis('equal')
Bbox = ((-118.75,-117.75,33.7,34.4))
la_image = plt.imread(file_path+"Los_Angeles_Open_Streetmap.png")
plt.title("Strategy 1 Geodesic Distance Static")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.xlim(Bbox[0],Bbox[1])
plt.ylim(Bbox[2],Bbox[3])
plt.imshow(la_image,zorder=0,extent=Bbox,aspect='equal')
plt.show()

## Strategy 2 

In [ ]:
vertex_seq = tri_g_trimmed.vs()
np.random.seed(2022)
frequencies = np.random.randint(1,1001, (len(vertex_seq),len(vertex_seq)) )
frequencies = (frequencies+frequencies.T)/2

In [ ]:
difference_frequency = np.multiply(extra_distance,frequencies)

In [ ]:
top_20_indices = k_largest_index_argsort(difference_frequency,40)


In [ ]:
index_list = set()
count = 0
while len(index_list)<20:
    i = top_20_indices[count]
    x = max(i[0],i[1])
    y = min(i[0],i[1])
    index_list.add((x,y))
    count+=1

index_list = list(index_list)
for i in index_list:
  print(i)

In [ ]:
tri_with_new_edges_v2 = deepcopy(tri_g_trimmed)
for i in index_list:
    tri_with_new_edges_v2.add_edges([(i[0],i[1])])

In [ ]:
# plot the roads and highlight our new edges. 
plt.figure(figsize=(15,15), dpi=200)
for e in tqdm(tri_with_new_edges_v2.es):
    start = e.source
    end = e.target
    cur_edge = (max(start,end),min(start,end))

    v1 = np.array(tri_with_new_edges_v2.vs(start)['coordinates'])
    v2 = np.array(tri_with_new_edges_v2.vs(end)['coordinates'])
    data = np.vstack([v1,v2])
    x = data[:, 0]
    y = data[:, 1]
    if cur_edge in index_list:
        plt.plot(x,y,'red') 
    else:
        plt.plot(x, y, 'blue')

plt.axis('equal')
Bbox = ((-118.75,-117.75,33.7,34.4))
la_image = plt.imread(file_path+"Los_Angeles_Open_Streetmap.png")
plt.title("Strategy 2 Geo Distance Static with Frequency")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.xlim(Bbox[0],Bbox[1])
plt.ylim(Bbox[2],Bbox[3])
plt.imshow(la_image,zorder=0,extent=Bbox,aspect='equal')
plt.show()

## Strategy 3 

In [ ]:
tri_g_with_new_edges_v3 = deepcopy(tri_g_trimmed)
budget = 20
new_dynamic_edges = []
node_list = list(tri_g_with_new_edges_v3.vs())
difference = normal_distance - euclid_distance 
top_one = k_largest_index_argsort(difference,1)[0]
new_dynamic_edges.append(top_one)
cost = euclid_distance[top_one[0],top_one[1]]
tri_g_with_new_edges_v3.add_edges([(top_one[0],top_one[1])],{"euclidean_cost":cost})
for i in tqdm(range(1,budget)):
    node_list=  list(tri_g_with_new_edges_v3.vs())
    euclid_distance = get_euclidean_distance(node_list)
    normal_distance = get_distance(tri_g_with_new_edges_v3)
    difference = normal_distance - euclid_distance 
    top_one = k_largest_index_argsort(difference,1)[0]
    new_dynamic_edges.append(top_one)
    cost = euclid_distance[top_one[0],top_one[1]]
    tri_g_with_new_edges_v3.add_edges([(top_one[0],top_one[1])],{"euclidean_cost":cost})

In [ ]:
index_list = set()
count = 0
while len(index_list)<20:
    i = new_dynamic_edges[count]
    x = max(i[0],i[1])
    y = min(i[0],i[1])
    index_list.add((x,y))
    count+=1

index_list = list(index_list)
for i in index_list:
  print(i)

In [ ]:
# plot the roads and highlight our new edges. 
plt.figure(figsize=(15,15), dpi=200)
for e in tqdm(tri_g_with_new_edges_v3.es):
    start = e.source
    end = e.target
    cur_edge = (max(start,end),min(start,end))

    v1 = np.array(tri_g_with_new_edges_v3.vs(start)['coordinates'])
    v2 = np.array(tri_g_with_new_edges_v3.vs(end)['coordinates'])
    data = np.vstack([v1,v2])
    x = data[:, 0]
    y = data[:, 1]
    if cur_edge in index_list:
        plt.plot(x,y,'red') 
    else:
        plt.plot(x, y,'blue')

plt.axis('equal')
Bbox = ((-118.75,-117.75,33.7,34.4))
la_image = plt.imread(file_path+"Los_Angeles_Open_Streetmap.png")
plt.title("Geodesic distance Dynamic")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.xlim(Bbox[0],Bbox[1])
plt.ylim(Bbox[2],Bbox[3])
plt.imshow(la_image,zorder=0,extent=Bbox,aspect='equal')
plt.show()
plt.show()

#### Strategy 4 

In [ ]:
def get_shortest_time(graph):
    vertex_seq = list(graph.vs())
    time = np.zeros((len(vertex_seq),len(vertex_seq)))
    time_weights = np.array([i['weight'] for i in graph.es()])
    counter_1 = 0
    counter_2  = 0
    for node1 in vertex_seq: 
        all_paths=  graph.get_shortest_paths(node1,vertex_seq,weights=graph.es['weight'],output='epath')
        counter_2 = 0
        for route in all_paths:
            local_time=np.sum(time_weights[route])
            time[counter_1,counter_2]=local_time
            counter_2+=1         
        counter_1+=1
    return time

In [ ]:
euclid_distance = get_euclidean_distance(list(tri_g_trimmed.vs()))
shortest_path_distance = get_distance(tri_g_trimmed)
travel_time_of_shortest_path=get_shortest_time(tri_g_trimmed)
travel_speed = shortest_path_distance/travel_time_of_shortest_path 
np.fill_diagonal(travel_speed,0)

extra_time = travel_time_of_shortest_path - euclid_distance/travel_speed
np.fill_diagonal(extra_time,0)

In [ ]:
shortest_path_distance.shape

In [ ]:
travel_time_of_shortest_path.shape

In [ ]:
top_20_indices = k_largest_index_argsort(extra_time,40)

In [ ]:
index_list = set()
count = 0
while len(index_list)<20:
    i = top_20_indices[count]
    x = max(i[0],i[1])
    y = min(i[0],i[1])
    index_list.add((x,y))
    count+=1

index_list = list(index_list)
for i in index_list:
  print(i)

In [ ]:
tri_with_new_edges_v4 = deepcopy(tri_g_trimmed)
for i in index_list:
    tri_with_new_edges_v4.add_edges([(i[0],i[1])])

In [ ]:
# plot the roads and highlight our new edges. 
plt.figure(figsize=(15,15), dpi=200)
for e in tqdm(tri_with_new_edges_v4.es):
    start = e.source
    end = e.target
    cur_edge = (max(start,end),min(start,end))

    v1 = np.array(tri_with_new_edges_v4.vs(start)['coordinates'])
    v2 = np.array(tri_with_new_edges_v4.vs(end)['coordinates'])
    data = np.vstack([v1,v2])
    x = data[:, 0]
    y = data[:, 1]
    if cur_edge in index_list:
        plt.plot(x,y,'red') 
    else:
        plt.plot(x, y,'blue')
plt.axis('equal')
Bbox = ((-118.75,-117.75,33.7,34.4))
la_image = plt.imread(file_path+"Los_Angeles_Open_Streetmap.png")
plt.title("Travel Time Static")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.xlim(Bbox[0],Bbox[1])
plt.ylim(Bbox[2],Bbox[3])
plt.imshow(la_image,zorder=0,extent=Bbox,aspect='equal')
plt.show()
plt.show()

#### Strategy 5

In [ ]:
tri_g_with_new_edges_v5 = deepcopy(tri_g_trimmed)
budget = 20
new_dynamic_edges = []
travel_speed = shortest_path_distance/travel_time_of_shortest_path 
euclid_speed = euclid_distance/travel_speed
extra_time = travel_time_of_shortest_path - euclid_speed
np.fill_diagonal(extra_time,0)

top_one = k_largest_index_argsort(extra_time,1)[0]
new_dynamic_edges.append(top_one)
time_cost = euclid_speed[top_one[0],top_one[1]]
distance_cost = euclid_distance[top_one[0],top_one[1]]
tri_g_with_new_edges_v5.add_edges([(top_one[0],top_one[1])],{"weight":time_cost,"euclidean_cost":distance_cost})

for i in tqdm(range(1,budget)):
    node_list = list(tri_g_with_new_edges_v5.vs())
    euclid_distance = get_euclidean_distance(node_list)
    shortest_path_distance= get_distance(tri_g_with_new_edges_v5)
    travel_time_of_shortest_path = get_shortest_time(tri_g_with_new_edges_v5)
    travel_speed = shortest_path_distance/travel_time_of_shortest_path 

    euclid_speed = euclid_distance/travel_speed
    extra_time = travel_time_of_shortest_path - euclid_speed
    np.fill_diagonal(extra_time,0)
    top_one = k_largest_index_argsort(extra_time,1)[0]
    new_dynamic_edges.append(top_one)
    time_cost =euclid_speed[top_one[0],top_one[1]]
    distance_cost = euclid_distance[top_one[0],top_one[1]]
    tri_g_with_new_edges_v5.add_edges([(top_one[0],top_one[1])],{"euclidean_cost":distance_cost,"weight":time_cost})


In [ ]:
index_list = set()
count = 0
while len(index_list)<20:
    i = new_dynamic_edges[count]
    x = max(i[0],i[1])
    y = min(i[0],i[1])
    index_list.add((x,y))
    count+=1

index_list = list(index_list)
for i in index_list:
  print(i)

In [ ]:
# plot the roads and highlight our new edges. 
plt.figure(figsize=(15,15), dpi=200)
for e in tqdm(tri_g_with_new_edges_v5.es):
    start = e.source
    end = e.target
    cur_edge = (max(start,end),min(start,end))

    v1 = np.array(tri_g_with_new_edges_v5.vs(start)['coordinates'])
    v2 = np.array(tri_g_with_new_edges_v5.vs(end)['coordinates'])
    data = np.vstack([v1,v2])
    x = data[:, 0]
    y = data[:, 1]
    if cur_edge in index_list:
        plt.plot(x,y,'red') 
    else:
        plt.plot(x, y, 'blue')
plt.axis('equal')
Bbox = ((-118.75,-117.75,33.7,34.4))
la_image = plt.imread(file_path+"Los_Angeles_Open_Streetmap.png")
plt.title("Travel Time Dynamic")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.xlim(Bbox[0],Bbox[1])
plt.ylim(Bbox[2],Bbox[3])
plt.imshow(la_image,zorder=0,extent=Bbox,aspect='equal')
plt.show()
plt.show()

## Strategy 6 

In [ ]:
np.random.seed(2022)
frequencies = np.random.randint(1,1001, (len(vertex_seq),len(vertex_seq)) )
frequencies = (frequencies+frequencies.T)/2
difference_frequency = np.multiply(extra_time,frequencies)

In [ ]:
top_20_indices = k_largest_index_argsort(difference_frequency,40)
index_list = set()
count = 0
while len(index_list)<20:
    i = top_20_indices[count]
    x = max(i[0],i[1])
    y = min(i[0],i[1])
    index_list.add((x,y))
    count+=1

index_list = list(index_list)
for i in index_list:
  print(i)

In [ ]:
tri_with_new_edges_v6 = deepcopy(tri_g_trimmed)
for i in index_list:
    tri_with_new_edges_v6.add_edges([(i[0],i[1])])

In [ ]:
# plot the roads and highlight our new edges. 
plt.figure(figsize=(15,15), dpi=200)
for e in tqdm(tri_with_new_edges_v6.es):
    start = e.source
    end = e.target
    cur_edge = (max(start,end),min(start,end))

    v1 = np.array(tri_with_new_edges_v6.vs(start)['coordinates'])
    v2 = np.array(tri_with_new_edges_v6.vs(end)['coordinates'])
    data = np.vstack([v1,v2])
    x = data[:, 0]
    y = data[:, 1]
    if cur_edge in index_list:
        plt.plot(x,y,'red') 
    else:
        plt.plot(x, y, 'blue')

plt.axis('equal')
Bbox = ((-118.75,-117.75,33.7,34.4))
la_image = plt.imread(file_path+"Los_Angeles_Open_Streetmap.png")
plt.title("Strategy 2 Geo Distance Static with Frequency")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.xlim(Bbox[0],Bbox[1])
plt.ylim(Bbox[2],Bbox[3])
plt.imshow(la_image,zorder=0,extent=Bbox,aspect='equal')
plt.show()